In [ ]:
from original_code import *

In [8]:
# Создаем тестовую задачу
W, D = create_sample_problem()

# Инициализируем решатель
utrp_solver = EnhancedUTRP(
    weight_matrix=W,
    demand_matrix=D,
    min_vertices=3,
    max_vertices=8,
    num_routes=5
)


In [9]:
print("Запуск DBMOSA...")
dbmosa_solution, dbmosa_att, dbmosa_trt = utrp_solver.dbmosa(
    max_iterations=500)
print(f"DBMOSA Результат: ATT={dbmosa_att:.2f}, TRT={dbmosa_trt:.2f}")


Запуск DBMOSA...
Генерация 5 кратчайших путей для всех пар вершин...
Сгенерировано 10 кандидатов в маршруты
DBMOSA Iteration 100: Temp=0.2635, Archive size=1, Best ATT=19.34
DBMOSA Iteration 200: Temp=0.0016, Archive size=1, Best ATT=19.34
DBMOSA Iteration 300: Temp=0.0000, Archive size=1, Best ATT=19.34
DBMOSA Iteration 400: Temp=0.0000, Archive size=1, Best ATT=19.34
DBMOSA Результат: ATT=19.34, TRT=147.09


In [10]:
print("\nЗапуск NSGA-II...")
nsga_front = utrp_solver.nsga_ii(population_size=30, generations=50)
if nsga_front:
    best_nsga = min(nsga_front, key=lambda x: (x[1], x[2]))
    print(
        f"NSGA-II Лучшее: ATT={best_nsga[1]:.2f}, TRT={best_nsga[2]:.2f}")
    print(f"Размер Pareto front: {len(nsga_front)}")

print("\nСтатистика операторов:")
for llh_name in utrp_solver.operators.keys():
    apps = utrp_solver.llh_applications[llh_name]
    success = utrp_solver.llh_successes[llh_name]
    success_rate = success / apps if apps > 0 else 0
    print(f"Оператор {llh_name}: Применений={apps}, Успехов={success}, "
          f"Успешность={success_rate:.2f}")



Запуск NSGA-II...
Генерация 5 кратчайших путей для всех пар вершин...
Сгенерировано 10 кандидатов в маршруты
NSGA-II Generation 0: Best compromise: ATT=19.11, TRT=191.00 | Front: ATT=[19.11-19.43], TRT=[178.49-191.00], size=3
NSGA-II Generation 10: Best compromise: ATT=17.78, TRT=194.12 | Front: ATT=[17.78-20.50], TRT=[139.62-194.12], size=16
NSGA-II Generation 20: Best compromise: ATT=17.47, TRT=202.02 | Front: ATT=[17.47-20.95], TRT=[151.59-202.02], size=13
NSGA-II Generation 30: Best compromise: ATT=17.58, TRT=203.15 | Front: ATT=[17.58-18.89], TRT=[155.53-203.15], size=15
NSGA-II Generation 40: Best compromise: ATT=17.74, TRT=181.99 | Front: ATT=[17.74-19.54], TRT=[148.68-181.99], size=15
NSGA-II Лучшее: ATT=17.61, TRT=191.96
Размер Pareto front: 13

Статистика операторов:
Оператор a: Применений=13, Успехов=4, Успешность=0.31
Оператор b: Применений=58, Успехов=48, Успешность=0.83
Оператор c: Применений=90, Успехов=82, Успешность=0.91
Оператор d: Применений=52, Успехов=46, Успешнос

In [11]:
# ========== ОПТИМИЗАЦИЯ ЧАСТОТ ==========
print("\n" + "="*60)
print("ОПТИМИЗАЦИЯ ЧАСТОТ МАРШРУТОВ")
print("="*60)

# Используем лучшее решение из DBMOSA
print(f"\nОптимизация частот для DBMOSA решения...")

# Оценка с оптимизацией частот
freq_result = utrp_solver.evaluate_with_frequency_optimization(
    dbmosa_solution,
    w_att=1.0,      # вес для времени пассажиров
    w_cost=0.01     # вес для операционных затрат
)


ОПТИМИЗАЦИЯ ЧАСТОТ МАРШРУТОВ

Оптимизация частот для DBMOSA решения...


### Выводы

In [12]:
print(f"\nБазовые частоты (все маршруты по {utrp_solver.default_frequency} рейсов/час):")
print(f"  ATT = {freq_result['att_baseline']:.2f} мин")
print(f"  Operating Cost = {freq_result['cost_baseline']:.2f}")

print(f"\nОптимальные частоты:")
for i, freq in enumerate(freq_result['frequencies']):
    interval = 60 / freq if freq > 0 else float('inf')
    print(
        f"  Маршрут {i}: {freq:.2f} рейсов/час (интервал {interval:.1f} мин)")

print(f"\nПосле оптимизации:")
print(f"  ATT = {freq_result['att']:.2f} мин")
print(f"  Operating Cost = {freq_result['operating_cost']:.2f}")
print(f"\n  Улучшение ATT: {freq_result['att_improvement_percent']:.2f}%")
print(f"  Изменение затрат: {freq_result['cost_change_percent']:.2f}%")



Базовые частоты (все маршруты по 5.0 рейсов/час):
  ATT = 19.34 мин
  Operating Cost = 1470.93

Оптимальные частоты:
  Маршрут 0: 1.00 рейсов/час (интервал 60.0 мин)
  Маршрут 1: 1.00 рейсов/час (интервал 60.0 мин)
  Маршрут 2: 1.00 рейсов/час (интервал 60.0 мин)
  Маршрут 3: 1.00 рейсов/час (интервал 60.0 мин)
  Маршрут 4: 4.96 рейсов/час (интервал 12.1 мин)

После оптимизации:
  ATT = 19.36 мин
  Operating Cost = 478.51

  Улучшение ATT: -0.11%
  Изменение затрат: -67.47%
